In [39]:
from itertools import chain, combinations
import numpy as np # Fundamental package for scientific computing with Python
import pandas as pd
import calendar
import time
from sklearn.metrics import accuracy_score, precision_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [40]:
# Train the model
def trainModel(X_train, y_train):
    # print(X_train)
    # Set regularization rate
    reg = 0.01
    # train a logistic regression model on the training set
    mod = LogisticRegression(C=1/reg, solver="liblinear", random_state=42).fit(X_train, y_train)

    return mod



In [41]:
def custom_predict(X, threshold, model):
    probs = model.predict_proba(X) 
    return (probs[:, 1] > threshold).astype(int)
# Definisci una funzione che applica la logica descritta
def calculate_gain(row, threshold,quota_minima):
    if(row['predictions'] > threshold):
        if row['Cluster'] == row['predictions']:
            if row['predictions'] == 1 and row['B365H'] > quota_minima:
                return row['B365H']-1
            elif row['predictions'] == 2 and row['B365A'] > quota_minima:
                return row['B365A']-1
            elif row['predictions'] == 0 and row['B365D'] > quota_minima:
                return row['B365D']-1
        else:
            if((row['predictions'] == 0 and row['B365D'] > quota_minima) or (row['predictions'] == 2 and row['B365A'] > quota_minima) or (row['predictions'] == 1 and row['B365H'] > quota_minima)):
                return -1

In [42]:
dataframe = pd.DataFrame(columns=['DataSetName', 'TrainAccuracy', 'TrainPrecision', 'TrainF2', 'TrainRecall',
                                  'TrainROC', 'TestAccuracy', 'TestPrecision', 'TestF2', 'TestRecall', 'TestROC', 'BestParams'])

features = ['HomeGoalsCumulative',	'AwayGoalsCumulative',	'HomePointsCumulative',
            'AwayPointsCumulative',	'HomeGoalsConcededCumulative',	'AwayGoalsConcededCumulative'
            # , 'B365H'
            ,	'B365D'
            # ,	'B365A'
            ]

combinazioni = chain.from_iterable(combinations(
    features, r) for r in range(1, len(features) + 1))

combos = []

for idx, combinazione in enumerate(combinazioni):
    current_GMT = time.gmtime()
    time_stamp = calendar.timegm(current_GMT)

    print(idx, time.strftime("%H:%M:%S", time.localtime()), ':', combinazione)
    combos.append(combinazione)

print(combos)

data = pd.read_excel('data/merged.xlsx')
# data['Cluster'] = np.where(data['FTHG'] == data['FTAG'], 1, 0)
data['Cluster'] = data['FTR'].map({'D': 0, 'H': 1, 'A': 2}) #np.where(data['FTHG'] == data['FTAG'], 1, 0)
data = data[data['B365H'].notna()]
data = data[data['B365A'].notna()]
data = data[data['B365D'].notna()]

for threshold in range(5, 70, 5):
    threshold /= 100
    
    print("Extracting features columns and creating target variable...")

    for idx, combo in enumerate(combos):
        current_GMT = time.gmtime()
        time_stamp = calendar.timegm(current_GMT)
        print(idx, time.strftime("%H:%M:%S", time.localtime()),
                'Trying with following combo', combo)
        feat_cols = [col for col in combo]

        x_train, x_test, y_train, y_test = train_test_split(
            data[feat_cols], data['Cluster'], test_size=0.3, random_state=42, shuffle=True)

        x_train = x_train.sort_index()
        x_test = x_test.sort_index()
        y_train = y_train.sort_index()
        y_test = y_test.sort_index()


        model = trainModel(x_train, y_train)

        # Predictions
        y_train_pred = model.predict(x_train.sort_index())
        # y_test_pred = model.predict(x_test.sort_index())
        text_pred_prob = model.predict_proba(x_test)
        # print('y_test_pred', y_test_pred)
        # clf.fit(x_train,y_train)

        # y_train_pred = clf.predict(x_train)
        # y_test_pred = clf.predict(x_test)

        # VN Calcolo la equity

        test_data = data[data.index.isin(x_test.index)]
        # test_data['prediction'] = y_test_pred
        test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
        # print('test_data[predictions]', test_data['predictions'])
        # test_data['gain'] = np.where((test_data['FTR'] == 'D') & (test_data['predictions'] > threshold),
        #                                 test_data['B365D']-1, np.where((test_data['FTR'] != 'D') & (test_data['predictions'] > threshold), -1, 0))
        test_data['gain'] = test_data.apply(calculate_gain, axis=1, args=(threshold,1.7,))
        test_equity = test_data['gain'].sum()
        # print('test_data',test_data)

        print("Train metrics...")
        # Metrics
        # train_accuracy = accuracy_score(y_train, y_train_pred)
        # train_precision = precision_score(y_train, y_train_pred)

        print("Test metrics...")
        test_accuracy = accuracy_score(y_test, test_data['predictions'])
        # test_precision = precision_score(y_test, y_test_pred)
        # test_f1 = f1_score(y_test, y_test_pred)

        # Save Results
        new_row = pd.Series({
            'DataSetName': str(combo) + str(threshold),
            # 'TrainAccuracy': train_accuracy,
            # 'TrainPrecision': train_precision,
            'TestAccuracy': test_accuracy,
            # 'TestPrecision': test_precision,
            # 'TestF1': test_f1,
            'BestParams': model.get_params,
            'TestEquity': test_equity,
            # 'TestCagr': test_cagr,
            # 'TestMaxDD': test_max_dd,
            # 'Eseguiti': numOp
        })
        print(new_row)
        dataframe = pd.concat(
            [dataframe, new_row.to_frame().T], ignore_index=True)

dataframe.to_excel("Dataframe/"+str(time_stamp) + "all.xlsx")

# print(result)

0 22:23:23 : ('HomeGoalsCumulative',)
1 22:23:23 : ('AwayGoalsCumulative',)
2 22:23:23 : ('HomePointsCumulative',)
3 22:23:23 : ('AwayPointsCumulative',)
4 22:23:23 : ('HomeGoalsConcededCumulative',)
5 22:23:23 : ('AwayGoalsConcededCumulative',)
6 22:23:23 : ('B365D',)
7 22:23:23 : ('HomeGoalsCumulative', 'AwayGoalsCumulative')
8 22:23:23 : ('HomeGoalsCumulative', 'HomePointsCumulative')
9 22:23:23 : ('HomeGoalsCumulative', 'AwayPointsCumulative')
10 22:23:23 : ('HomeGoalsCumulative', 'HomeGoalsConcededCumulative')
11 22:23:23 : ('HomeGoalsCumulative', 'AwayGoalsConcededCumulative')
12 22:23:23 : ('HomeGoalsCumulative', 'B365D')
13 22:23:23 : ('AwayGoalsCumulative', 'HomePointsCumulative')
14 22:23:23 : ('AwayGoalsCumulative', 'AwayPointsCumulative')
15 22:23:23 : ('AwayGoalsCumulative', 'HomeGoalsConcededCumulative')
16 22:23:23 : ('AwayGoalsCumulative', 'AwayGoalsConcededCumulative')
17 22:23:23 : ('AwayGoalsCumulative', 'B365D')
18 22:23:23 : ('HomePointsCumulative', 'AwayPointsCumu

C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = test_data.apply(calculate_gain, axis=1, args=(threshold,1.7,))
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on

Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'AwayGoalsCumulative')...
TestAccuracy                                             0.412335
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                -124.37
dtype: object
8 22:23:26 Trying with following combo ('HomeGoalsCumulative', 'HomePointsCumulative')
Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'HomePointsCumulative'...
TestAccuracy                                             0.412335
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                -125.37
dtype: object
9 22:23:26 Trying with following combo ('HomeGoalsCumulative', 'AwayPointsCumulative')
Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'AwayPointsCumulative'...
TestAccuracy                                             0.412335
BestParams      <bound method BaseEstim

C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = test_data.apply(calculate_gain, axis=1, args=(threshold,1.7,))
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on

Train metrics...
Test metrics...
DataSetName     ('AwayGoalsCumulative', 'HomeGoalsConcededCumu...
TestAccuracy                                             0.412335
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                -125.37
dtype: object
16 22:23:26 Trying with following combo ('AwayGoalsCumulative', 'AwayGoalsConcededCumulative')
Train metrics...
Test metrics...
DataSetName     ('AwayGoalsCumulative', 'AwayGoalsConcededCumu...
TestAccuracy                                             0.412335
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                -125.37
dtype: object
17 22:23:27 Trying with following combo ('AwayGoalsCumulative', 'B365D')
Train metrics...
Test metrics...
DataSetName                  ('AwayGoalsCumulative', 'B365D')0.05
TestAccuracy                                             0.412335
BestParams      <bound method BaseEstimator.g

...

Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'HomePointsCumulative'...
TestAccuracy                                             0.321586
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                    0.8
dtype: object
112 22:24:35 Trying with following combo ('HomeGoalsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative', 'AwayGoalsConcededCumulative', 'B365D')
Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'AwayPointsCumulative'...
TestAccuracy                                             0.323348
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                   1.88
dtype: object
113 22:24:35 Trying with following combo ('AwayGoalsCumulative', 'HomePointsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative', 'AwayGoalsConcededCumulative')
Train metrics...
Test metrics...
DataSetNam

C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = test_data.apply(calculate_gain, axis=1, args=(threshold,1.7,))
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on

Train metrics...
Test metrics...
DataSetName     ('AwayGoalsCumulative', 'HomePointsCumulative'...
TestAccuracy                                             0.324229
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                  -0.27
dtype: object
117 22:24:35 Trying with following combo ('AwayGoalsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative', 'AwayGoalsConcededCumulative', 'B365D')
Train metrics...
Test metrics...
DataSetName     ('AwayGoalsCumulative', 'AwayPointsCumulative'...
TestAccuracy                                              0.32511
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                      0
dtype: object
118 22:24:35 Trying with following combo ('HomePointsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative', 'AwayGoalsConcededCumulative', 'B365D')
Train metrics...
Test metrics...
DataSetName     ('HomePo

C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predictions'] = custom_predict(x_test.sort_index(), threshold, model)
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['gain'] = test_data.apply(calculate_gain, axis=1, args=(threshold,1.7,))
C:\Users\vitto\AppData\Local\Temp\ipykernel_1492\2600751362.py:69: SettingWithCopyWarning: 
A value is trying to be set on

Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'AwayGoalsCumulative',...
TestAccuracy                                             0.320705
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                   0.78
dtype: object
123 22:24:35 Trying with following combo ('HomeGoalsCumulative', 'AwayGoalsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative', 'AwayGoalsConcededCumulative', 'B365D')
Train metrics...
Test metrics...
DataSetName     ('HomeGoalsCumulative', 'AwayGoalsCumulative',...
TestAccuracy                                             0.322467
BestParams      <bound method BaseEstimator.get_params of Logi...
TestEquity                                                   0.78
dtype: object
124 22:24:36 Trying with following combo ('HomeGoalsCumulative', 'HomePointsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative', 'AwayGoalsConcededCumulative', 'B365D')
Train metri